In [1]:
#install Required Packages
!pip install hmmlearn librosa scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 3.0 MB/s eta 0:00:00


In [2]:
#Import Libraries
import numpy as np
import librosa
import os
from hmmlearn import hmm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [14]:
from google.colab import files
import zipfile

uploaded = files.upload()

for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall("tones_dataset")


Saving tones_dataset.zip to tones_dataset (1).zip


In [15]:
#Feature extraction (MFCC)
def extract_features(file_path, n_mfcc=13):
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return mfcc.T

In [19]:
#Prepare Dataset
import glob
X = []
y = []

for file_path in glob.glob("tones_dataset/tones_dataset/**/*.wav", recursive=True):
    label = os.path.basename(os.path.dirname(file_path))  # Folder name as label (0-9)
    features = extract_features(file_path)
    X.append(features)
    y.append(label)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [20]:
#Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

In [22]:
#Train One HMM per Class (Number)
models = {}
num_states = 5

for label in np.unique(y_train):
    X_class = [X_train[i] for i in range(len(X_train)) if y_train[i] == label]
    X_concat = np.vstack(X_class)
    lengths = [len(seq) for seq in X_class]

    model = hmm.GaussianHMM(n_components=num_states, covariance_type='diag', n_iter=200, tol=0.01, verbose=True)
    model.fit(X_concat, lengths)
    models[label] = model

         1   -9752.14342557             +nan
         2   -8159.38059818   +1592.76282739
         3   -7486.01798214    +673.36261604
         4   -7427.05943298     +58.95854916
         5   -7424.77310063      +2.28633235
         6   -7411.87073735     +12.90236328
         7   -7391.18156983     +20.68916751
         8   -7377.32432268     +13.85724715
         9   -7350.32092785     +27.00339483
        10   -7237.34182037    +112.97910748
        11   -7100.55481322    +136.78700714
        12   -7043.33998929     +57.21482394
        13   -7043.28576865      +0.05422064
        14   -7043.28502074      +0.00074791
         1  -10055.16327933             +nan
         2   -7588.53875679   +2466.62452254
         3   -7312.64542053    +275.89333626
         4   -7254.04112058     +58.60429994
         5   -7187.89353573     +66.14758485
         6   -6997.26570592    +190.62782981
         7   -6846.76475361    +150.50095231
         8   -6846.21432977      +0.55042384
         9

In [25]:
import joblib
joblib.dump(models, 'hmm_models.pkl')  # Save the trained models to a file


['hmm_models.pkl']

In [23]:
#Predict
correct = 0

for i in range(len(X_test)):
    scores = {}
    for label, model in models.items():
        scores[label] = model.score(X_test[i])

    predicted_label = max(scores, key=scores.get)
    if predicted_label == y_test[i]:
        correct += 1

accuracy = correct / len(X_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")

Test Accuracy: 96.67%


In [24]:
#Report
print("Classes (keypad numbers):", label_encoder.classes_)

Classes (keypad numbers): ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9']


In [26]:
from google.colab import files
files.download('hmm_models.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>